In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 16), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [3]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [4]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.3MB 45.6MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-18 04:06:11--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.1, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.0|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?

In [0]:
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1] is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle
import MeCab

tagger = MeCab.Tagger()


In [0]:
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()
        
        V = embed_num
        D = 100 #args.embed_dim
        C = class_num
        Ci = 1
        Co = 20 #args.kernel_num
        Ks = [1,2,3]

        self.embed = nn.Embedding(V, D)

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D), padding=(2,0)) for K in Ks])

        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        
        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        
        x = torch.cat(x, 1)

        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit

In [0]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in open('/content/gdrive/My Drive/Colab Notebooks/mydata2.txt','r',encoding='utf-8'):
    line = line.strip().split('\t')
    if len(line)>1:
        intent=line[1]
        if intent not in intent_vocab:
            intent_vocab[intent]=intent_idx
            intent_list.append(intent)
            intent_idx +=1
    else:
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            if it not in content_vocab:
                content_vocab[it] = vocab_idx
                vocab_idx +=1
                
                

In [10]:
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()

33 3


CNN_Text(
  (embed): Embedding(33, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1), padding=(2, 0))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=60, out_features=3, bias=True)
)

In [11]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    for line in open('/content/gdrive/My Drive/Colab Notebooks/mydata2.txt','r',encoding='utf-8'):
        line = line.strip().split('\t')
    
        if len(line)> 1:
            target = Variable(torch.LongTensor([intent_vocab[line[1]]]))
            continue

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            cont.append(content_vocab[it])
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)

        loss = F.cross_entropy(pred,target)
        totalloss += loss.data
        loss.backward()
        optimizer.step()
    print (totalloss)

tensor(91.1299)
tensor(29.5724)
tensor(3.5053)
tensor(1.3682)
tensor(0.5343)
tensor(0.5203)
tensor(0.4893)
tensor(0.5428)
tensor(0.2372)
tensor(0.3824)


In [0]:
response = []
for line in open('/content/gdrive/My Drive/Colab Notebooks/response2.txt','r',encoding='utf-8'):
    line=line.strip()
    response.append(line)

In [14]:
cnn.eval()
for line in open('/content/gdrive/My Drive/Colab Notebooks/testdata2.txt','r',encoding='utf-8'):
    line = line.strip()
    
    line = mecabsplit(tagger,line,False)
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it]) # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    print (torch.nn.functional.softmax(pred,dim=-1).data.numpy() )
    print('intent : ',intent_list[int(i)])
    print(response[int(i)])
    print()

input :  ['\ufeff', '동국', '대', '학사', '일정', '보이', '어', '주', '어']
[[2.9833693e-04 1.9113932e-04 9.9951053e-01]]
intent :  ASK_SCHEDULE
동국대학교의 최신 학사일정 표입니다.

input :  ['동국', '대', '맛', '집', '은', '?']
[[0.00109892 0.9972451  0.00165602]]
intent :  ASK_DINNING
동국대학교의 맛집은 닭한마리입니다.

input :  ['Dongguk', 'Univ', '소식', '알리', '어', '주', '어']
[[9.9948597e-01 2.8443907e-04 2.2959418e-04]]
intent :  ASK_NEWS
﻿동국대학교에 대한 최신 뉴스입니다.

